# Pangenome dataset export

## Setup

In [1]:
## Notebook config

import sys
if '../' not in sys.path:
    sys.path.append("../")
%load_ext dotenv
%reload_ext dotenv
%dotenv

# Notebook imports
import os
import ast

from graphdatascience import GraphDataScience
import pandas as pd

/home/ec2-user/miniconda3/envs/rnalab/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_DIR = './dataset/'
PROJECTION_NAME_PREFIX = 'dataset_projection_'

NEO4J_URI = os.environ.get('NEO4J_URI')
NEO4J_USER = os.environ.get('NEO4J_USER')
NEO4J_PASSWORD = os.environ.get('NEO4J_PASSWORD')

## Create GDS projection

In [5]:
# GDS utils

graph_config = {
    'node_spec': {
        'Fasta': {},
        'Hit': {},
        'HitFamily': {},
        'HitRegion': {},
    },
    'relationship_spec' : {
        'hasHit': {'orientation': 'NATURAL', 'properties': {'score': {'defaultValue': 1}}},
        'hasRegion': {'orientation': 'NATURAL', 'properties': {'score': {'defaultValue': 1}}},
        'hasAffiliate': {'orientation': 'NATURAL', 'properties': {'coverage': {'defaultValue': 1}}},
        'hasMember': {'orientation': 'NATURAL', 'properties': {'probab': {'defaultValue': 1}}},
        'hasDownstream': {'orientation': 'NATURAL', 'properties': {'score': {'defaultValue': 1}}},
    },
}

def get_pg_client():
    return GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def get_kg_connection():
    return GraphDataScience(
        os.environ.get('NEO4J_KG_URI'),
        auth=(
            os.environ.get('NEO4J_KG_USER'), 
            os.environ.get('NEO4J_KG_PASSWORD'),
        )
    )

def get_projection_name(version=1):
    return PROJECTION_NAME_PREFIX + str(version)

def get_projection(gds, projection_name):
    if gds.graph.exists(projection_name)['exists']:
        return gds.graph.get(projection_name)
    projection = gds.graph.project(
        graph_name=projection_name,
        node_spec=list(graph_config['node_spec'].keys()),
        relationship_spec=graph_config['relationship_spec'],
    )
    return projection

def delete_projection(gds, projection_name):
    if gds.graph.exists(projection_name)['exists']:
        gds.graph.drop(gds.graph.get(projection_name))

# At least one common node property is needed to use the gds node export function
def add_degree_property(gds, projection_name):
    df_degree = gds.degree.mutate(
        G=gds.graph.get(projection_name),
        mutateProperty='degree',
        nodeLabels=["*"],
        relationshipTypes=["*"],
    )

def add_fastrp_embedding(gds, projection_name):
    df_fastrp = gds.fastRP.mutate(
        G=gds.graph.get(projection_name),
        mutateProperty='fastrp',
        randomSeed=42,
        # relationshipWeightProperty='weight',
        nodeLabels=[
            'Fasta',
            'Hit',
            'HitFamily',
            'HitRegion',
        ],
        relationshipTypes=[
            "hasHit",
            "hasRegion",
            "hasAffiliate",
            "hasMember",
            "hasDownstream",
        ],
        embeddingDimension=128,
        normalizationStrength=-0.5,
        # nodeSelfInfluence=2,
    )

In [6]:
# Create GDS projection

version = 1
projection_name = get_projection_name(version)
gds = get_pg_client()
delete_projection(gds, projection_name)
projection = get_projection(gds, projection_name)
add_degree_property(gds, projection_name)
print(projection)

GraphCreateResult(graph=Graph({'graphName': 'dataset_projection_1', 'nodeCount': 148782, 'relationshipCount': 393619, 'database': 'neo4j', 'configuration': {'relationshipProjection': {'hasHit': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {'score': {'aggregation': 'DEFAULT', 'property': 'score', 'defaultValue': 1}}, 'type': 'hasHit'}, 'hasMember': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {'probab': {'aggregation': 'DEFAULT', 'property': 'probab', 'defaultValue': 1}}, 'type': 'hasMember'}, 'hasRegion': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {'score': {'aggregation': 'DEFAULT', 'property': 'score', 'defaultValue': 1}}, 'type': 'hasRegion'}, 'hasDownstream': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {'score': {'aggregation': 'DEFAULT', 'property': 'score', 'defaultValue': 1}}, 'type': 'hasDownstream

## Export Pangenome Datasets

In [7]:
NUMBER_OF_NODES = 148782
NUMBER_OF_EDGES = 393619

In [8]:
# Export nodes from projection to df

def export_nodes_from_projection(property):
    nodes = gds.graph.nodeProperties.stream(
        get_projection(gds, get_projection_name()),
        node_properties=[property],
        separate_property_columns=True,
        db_node_properties=[
            'accession',
            'name',
            'annotation',
            'subtype',
            'std_length',
            'source',
            'taxonomy',
            'circular',
            'begin',
            'end',
            'hit_begin'
            'hit_end',
            'frame',
            'strand',
        ],
        listNodeLabels=True,
    )
    nodes['nodeLabels'] = nodes['nodeLabels'].apply(lambda x: x[0])
    return nodes


nodes = export_nodes_from_projection('degree')

assert nodes.shape[0] == NUMBER_OF_NODES

/home/ec2-user/miniconda3/envs/rnalab/lib/python3.12/site-packages/graphdatascience/query_runner/neo4j_query_runner.py:222: RuntimeWarning: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: hit_beginhit_end)
  warnings.warn(warning)


In [19]:
# Enrich nodes df and save to csv

## Enrich Genomes with taxonomic labels from ICTV string
def get_taxonomy_by_idx(tax_str, idx):
    try:
        value = tax_str.split(';')[idx].strip()
        if value == '' or value == 'Null':
            return None
        return value
    except:
        return None

def enrich_data_with_tax_labels(df):
    rank_index = {
        'clade': 0,
        'kingdom': 2,
        'phylum': 4,
        'class': 6,
        'order': 8,
        'family': 10,
        'genus': 12,
        'species': 14,
    }
    for rank in ['species', 'genus', 'family', 'order', 'class', 'phylum']:
        df[rank] = None
        df.loc[df['nodeLabels'] == 'Fasta', rank] = df.loc[
            df['nodeLabels'] == 'Fasta', 'taxonomy'].apply(get_taxonomy_by_idx, args=[rank_index[rank]])
    return df

## nodeId is not stable in neo4j, instead add a reference to a stable node attribute
def set_node_app_id(df):
    config = {
        'Fasta': 'accession',
        'Hit': 'name',
        'HitFamily': 'accession',
        'HitRegion': 'name',
    }
    df['appId'] = None
    for node_label, id_column in config.items():
        df.loc[df['nodeLabels'] == node_label, 'appId'] = df.loc[
            df['nodeLabels'] == node_label, id_column]
    
    return df

## Add isSegmented to Fasta nodes
def set_segmented(df):
    df['isSegmented'] = None
    df.loc[df['nodeLabels'] == 'Fasta', 'isSegmented'] = df.loc[
        df['nodeLabels'] == 'Fasta', 'name'].apply(lambda x: '|Seg' in x)
    

enrich_data_with_tax_labels(nodes)
set_node_app_id(nodes)
set_segmented(nodes)

assert nodes.shape[0] == NUMBER_OF_NODES
nodes.to_csv(DATASET_DIR + 'pangenome_nodes.csv', index=False)

In [21]:
# Export dataset edges from projection

edges = pd.DataFrame()
for unique_property in ['score', 'coverage', 'probab']:
    edges = pd.concat([
        edges,
        gds.graph.relationshipProperties.stream(
            get_projection(gds, get_projection_name(version)),
            relationship_properties=[unique_property],
        )
    ])

# reset index
edges.reset_index(drop=True, inplace=True)

assert edges.shape[0] == NUMBER_OF_EDGES

In [23]:
# Enrich edges and save to csv

node_map_cache = {}
def set_edge_app_ids(df):
    df_copy = df.copy()
    df_copy['sourceAppId'] = None
    df_copy['targetAppId'] = None
    for i, row in df.iterrows():
        if row['sourceNodeId'] not in node_map_cache:
            node_map_cache[row['sourceNodeId']] = nodes.loc[nodes['nodeId'] == row['sourceNodeId'], 'appId'].values[0]
        if row['targetNodeId'] not in node_map_cache:
            node_map_cache[row['targetNodeId']] = nodes.loc[nodes['nodeId'] == row['targetNodeId'], 'appId'].values[0]

        df_copy.at[i, 'sourceAppId'] = node_map_cache[row['sourceNodeId']]
        df_copy.at[i, 'targetAppId'] = node_map_cache[row['targetNodeId']]

        # if row['sourceNodeId'] == 44762:
        #     # print(row['sourceNodeId'], node_map_cache[row['sourceNodeId']])
        #     print(df_copy.at[i, 'sourceAppId'])

        # if i > 100:
        #     # print()
        #     print(
        #         df_copy.loc[df_copy['sourceNodeId'] == 44762, 'sourceAppId'],
        #     )
        #     break

    return df_copy

def set_edge_weights(df):
    config = {
        'hasHit': 'score',
        'hasRegion': 'score',
        'hasAffiliate': 'coverage',
        'hasMember': 'probab',
        'hasDownstream': 'score',
    }
    df['weight'] = None
    for relationship_type, weight_column in config.items():
        condition = (df['relationshipType'] == relationship_type) & (df['relationshipProperty'] == weight_column)
        df.loc[condition, 'weight'] = df.loc[condition, 'propertyValue']
    return df

edges = set_edge_weights(edges)
edges = set_edge_app_ids(edges)

assert edges.shape[0] == NUMBER_OF_EDGES
edges.to_csv(DATASET_DIR + 'pangenome_edges.csv', index=False)

In [20]:
# drop projection from Neo4j DB memory
gds.graph.drop(get_projection_name())

graphName                                             dataset_projection_1
database                                                             neo4j
databaseLocation                                                     local
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                           148782
relationshipCount                                                   787238
configuration            {'relationshipProjection': {'hasHit': {'aggreg...
density                                                           0.000036
creationTime                           2024-04-30T23:10:29.592752103+00:00
modificationTime                       2024-04-30T23:10:31.124691843+00:00
schema                   {'graphProperties': {}, 'nodes': {'HitRegion':...
schemaWithOrientation    {'graphProperties': {}, 'nodes': {'HitRegion':...
Name: 0, dtype: object

## Export NCBI Taxonomy dataset

In [14]:
# Create taxonomy projection from existing serratus/SRA knowledge graph

taxonomy_projection_name = 'taxonomy_projection'

def get_taxon_nodes(gds):
    if os.path.exists(DATASET_DIR + 'taxon_nodes.csv'):
        return pd.read_csv(DATASET_DIR + 'taxon_nodes.csv')
    query = '''
            MATCH (n:Taxon)
            RETURN
                id(n) as nodeId,
                n.taxId as appId,
                n.taxId as taxId,
                labels(n) as nodeLabels,
                n.rank as rank,
                n.taxKingdom as taxKingdom,
                n.taxPhylum as taxPhylum,
                n.taxOrder as taxOrder,
                apoc.node.degree(n, "HAS_PARENT") as hasParentDegree,
                apoc.node.degree(n, "HAS_PARENT>") as hasParentOutDegree,
                apoc.node.degree(n, "<HAS_PARENT") as hasParentInDegree
            '''
    df = gds.run_cypher(query)
    df['nodeLabels'] = df['nodeLabels'].apply(lambda x: ast.literal_eval(x)[0])
    df.to_csv(DATASET_DIR + 'taxon_nodes.csv', index=False)
    return df


def get_taxon_has_parent_edges(gds):
    if os.path.exists(DATASET_DIR + 'taxon_has_parent_edges.csv'):
        return pd.read_csv(DATASET_DIR + 'taxon_has_parent_edges.csv')
    query = '''
            MATCH (s:Taxon)-[r:HAS_PARENT]->(t:Taxon)
            RETURN
                id(s) as sourceNodeId,
                s.taxId as sourceAppId,
                id(t) as targetNodeId,
                t.taxId as targetAppId,
                type(r) as relationshipType,
                1 as weight
            '''
    df = gds.run_cypher(query)
    df.to_csv(DATASET_DIR + 'taxon_has_parent_edges.csv', index=False)
    return df

kg_client = get_kg_connection()
taxon_nodes = get_taxon_nodes(kg_client)
taxon_has_parent_edges = get_taxon_has_parent_edges(kg_client)

In [21]:
# Export node embeddings for testing

def get_taxonomy_projection(gds, projection_name):
    if gds.graph.exists(projection_name)['exists']:
        return gds.graph.get(projection_name)
    nodes = get_taxon_nodes(gds)
    nodes = nodes.loc[
        nodes['taxKingdom'] == 'Viruses'
    ]
    nodes['labels'] = nodes['nodeLabels']
    nodes = nodes[[
        'nodeId',
        'labels',
    ]]

    relationships = get_taxon_has_parent_edges(gds)
    relationships = relationships[[
        'sourceNodeId',
        'targetNodeId',
        'relationshipType',
        'weight'
    ]]
    undirected_relationship_types = relationships['relationshipType'].unique().tolist()
    projection = gds.graph.construct(
        graph_name=projection_name,
        nodes=nodes,
        relationships=relationships,
        concurrency=4,
        undirected_relationship_types=undirected_relationship_types,
    )
    return projection

def delete_projection(gds, projection_name):
    if gds.graph.exists(projection_name)['exists']:
        gds.graph.drop(gds.graph.get(projection_name))

def add_fastrp_embedding(gds, projection_name):
    df_fastrp = gds.fastRP.mutate(
        G=gds.graph.get(projection_name),
        mutateProperty='fastrp',
        randomSeed=42,
        nodeLabels=[
            'Taxon',
        ],
        relationshipTypes=[
            "HAS_PARENT",
        ],
        embeddingDimension=128,
        normalizationStrength=-0.5,
    )

def export_nodes_from_projection(gds, projection_name, property):
    nodes = gds.graph.nodeProperties.stream(
        get_projection(gds, projection_name),
        node_properties=[property],
        separate_property_columns=True,
        db_node_properties=[
            'taxId',
            'rank',
            'taxKingdom',
            'taxPhylum',
            'taxOrder',
        ],
        listNodeLabels=True,
    )
    nodes['nodeLabels'] = nodes['nodeLabels'].apply(lambda x: x[0])
    return nodes


def set_node_app_id(df):
    config = {
        'Taxon': 'taxId',
    }
    df['appId'] = None
    for node_label, id_column in config.items():
        df.loc[df['nodeLabels'] == node_label, 'appId'] = df.loc[
            df['nodeLabels'] == node_label, id_column]
    return df


kg_client = get_kg_connection()
delete_projection(gds, projection_name)
projection = get_taxonomy_projection(kg_client, taxonomy_projection_name)
add_fastrp_embedding(kg_client, taxonomy_projection_name)

nodes = export_nodes_from_projection(kg_client, taxonomy_projection_name, 'fastrp')
nodes = set_node_app_id(nodes)
nodes.to_csv(DATASET_DIR + 'taxon_nodes_emb.csv', index=False)

In [37]:
# Create Fasta to Taxon relationships

def get_fasta_has_tax_order_edges(gds):
    if os.path.exists(DATASET_DIR + 'fasta_has_taxon_taxon_edges.csv'):
        return pd.read_csv(DATASET_DIR + 'fasta_has_taxon_taxon_edges.csv')

    taxon_nodes = pd.read_csv(DATASET_DIR + 'taxon_nodes.csv')
    order_nodes = taxon_nodes.loc[taxon_nodes['rank'] == 'order']

    pangenome_nodes = pd.read_csv(DATASET_DIR + 'pangenome_nodes.csv')
    fasta_nodes = pangenome_nodes.loc[pangenome_nodes['nodeLabels'] == 'Fasta']
    
    merged = pd.merge(
        fasta_nodes,
        order_nodes,
        how='left',
        left_on='order',
        right_on='taxOrder',
        suffixes=('', '_taxon')
    )
    
    print(merged['taxOrder'].isna().sum())

    rows = []
    for i, fasta in merged.iterrows():
        if pd.isna(fasta['taxOrder']):
            continue
        rows.append({
            'sourceNodeId': fasta['nodeId'],
            'targetNodeId': int(fasta['nodeId_taxon']),
            'sourceAppId': fasta['appId'],
            'targetAppId': int(fasta['appId_taxon']),
            'relationshipType': 'HAS_TAX_ORDER',
            'weight': 1,
        })
    df = pd.DataFrame(rows)
    df.to_csv(DATASET_DIR + 'fasta_has_tax_order_edges.csv', index=False)
    return df

fasta_has_tax_order_edges = get_fasta_has_tax_order_edges(kg_client)

120
